## 2、梯度计算

在模型的训练过程中，损失函数L对于权重W的梯度计算是核心。训练过程中，用的训练数据不变即输入X不变，权重W在更新，通常是求损失函数关于权重W的梯度而非关于输入X的梯度。梯度是全部变量的偏导数汇总而成的向量，本质是求导数，计算机实现导数计算有以下四种方法：
1. 手动微分（Manual Differentiation）
2. 符号微分（Symbolic Differentiation）
3. 数值微分（numerical differentiation）
4. 自动微分（Automatic Differentiation or auto-diff）

图片来源[Automatic differentiation in machine learning: a survey](https://arxiv.org/pdf/1502.05767.pdf)

<img src="./notebook-imgs/4-autodiff.png" width="600">

### 2.1 手动微分（Manual Differentiation）

手动根据函数和链式法则进行推倒，得到导数求解公式解后再编码实现。

### 2.2 符号微分（Symbolic Differentiation）
符号计算又称计算机代数，用计算机推导数学公式。符号微分，通过sympy等库将变量x看做符号，直接求出导数的解析解的形式（Analytic Gradient）。符号微分计算比较慢，需要知道函数的闭合形式（closed form），不支持if语句、while语句等。以下是求解上图中的f(x)导数的例子。

In [ ]:
# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%reload_ext autoreload
%autoreload 2
import sympy as sy  # # pip install sympy

x = sy.symbols('x')
L1 = x
L2 = 4*L1*(1-L1)
L3 = 4*L2*(1-L2)
L4 = 4*L3*(1-L3)
L4.simplify()
expr = L4.expand()
expr_diff = expr.diff(x, 1)  #或者 sympy.diff(expr, x) , 参数1表示求一阶导
print(expr_diff)

In [ ]:
# 计算函数f(x)在x=2处的函数值和导数值
print('f(2)  =',expr.evalf(subs={x:2}))
print('f\'(2) =',expr_diff.evalf(subs={x:2}))

再看一个例子，有函数$z=x*y+\sin(x)$，求$\tfrac{\partial z}{\partial x}$，即z关于x的偏导数，符号微分结果如下

In [ ]:
x = sy.symbols('x')
y = sy.symbols('y')
z = x*y+ sy.sin(x)
z_symbol_diff = z.diff(x, 1) 
print(z_symbol_diff)

In [ ]:
# 计算函数z=x*y+sin(x)在x=1,y=1处的函数值、z关于x的偏导值
print('funtion value at (x=1,y=1):      z =', z.evalf(subs={x:1,y:1}))
print('symbolic diff at (x=1,y=1):  dz/dx =',z_symbol_diff.evalf(subs={x:1,y:1}))

### 2.3 数值微分（numerical differentiation）

从导数定义的衍生而来，利用微小的差分求导数，以下是导数公式，由于计算机无法表示无穷小，计算的时候h无法趋近于0，而是取一个比较小的值，通常取1e−4或1e-5等。

$$\text{ }\ \tfrac{df(x)}{x}= \lim_{h \to 0}\tfrac{df(x+h)-f(x)}{h}$$

In [ ]:
h = 1e-4 # 0.0001
def numerical_diff_forward(f, x):
    return (f(x+h) - f(x)) / h

数值微分算出来的是估计值，存在偏差。为了减小这个误差，可以计算函数f在(x + h)和(x − h)之间的差分。因为这种计算方法以x为中心算左右两边的差，也称为中心差分，而(x + h)和x之间的差分为前向差分。

数值微分主要的问题是误差不好控制，误差来源：浮点计算存在误差；差值h过小会有舍入误差，过大会不准确。再就是计算复杂度高，如果要求n个变量的偏导，计算复杂度是O(n)，即n次全量的前向计算。数值微分常用于自动微分结果正确性的校验。

In [ ]:
def numerical_diff_center(f, x):
    return (f(x+h) - f(x-h)) / (2*h)

In [ ]:
# f(x)=L4=64*x*(1−x)*(1−2*x)^2*(1-8*x+8*x^2)^2
def f(x):
    v = x
    for i in [1, 2, 3]:
        v = 4*v*(1-v)
    return v

print('function value        : f(2) =', f(2))
print('numerical_diff_forward: f\'(2)=',numerical_diff_forward(f, 2))
print('numerical_diff_center : f\'(2)=', numerical_diff_center(f, 2))

In [ ]:
import numpy as np
def numerical_gradient_2d(f, x, y):
    grad = np.zeros_like(x)
    grad_x =  (f(x+h,y) - f(x-h, y)) / (2*h)
    grad_y =  (f(x,y+h) - f(x, y-h)) / (2*h)
    return grad_x, grad_y

# z=x*y+sin(x)
def f_z(x, y):
    return  x*y+ np.sin(x)  # np.sin: x, Angle, in radians (rad equals 360 degrees).
    
x = 1
y = 1

# 用数值微分的方式计算x=1,y=1时z关于x的偏导数的值
print('func value at (x=1,y=1)    :      z =', f_z(x, y))
print('numerical diff at (x=1,y=1):  dz/dx =',numerical_gradient_2d(f_z, x, y)[0])

### 2.4 自动微分（Automatic Differentiation or auto-diff）

根据导数的链式法则进行计算，具体实现有前向（forward mode）和后向（reverse mode）两种模式，以下面公式为例，y是输出，x是输入，w1,w2,w3都是中间变量。

$$\text{ }\ y=f(g(h(x)))=f(g(h(w0)))=f(g(w1))=f(w2)=w3$$

其中：

$
\begin{aligned}
& w0=x \\
& w1=h(w0) \\
& w2=g(w1) \\
& w3=f(w2)=y \\
\end{aligned}
$


y关于x的导数，根据链式法则有
$$\tfrac{\partial y}{\partial x} =\tfrac{\partial y}{\partial w2} \tfrac{\partial w2}{\partial w1} \tfrac{\partial w1}{\partial x} $$

计算顺序（结合顺序）有两种：

（1）前向，从输入节点开始，先计算$\tfrac{\partial w1}{\partial x}$，再计算$\tfrac{\partial w2}{\partial w1}$并乘以前面的结果，再计算$\tfrac{\partial y}{\partial w2}$并乘以前面的结果，即$\tfrac{\partial y}{\partial w2} (\tfrac{\partial w2}{\partial w1} (\tfrac{\partial w1}{\partial x}))$

（2）反向，从输出节点开始，先计算$\tfrac{\partial y}{\partial w2}$，再计算$\tfrac{\partial w2}{\partial w1}$并乘以前面的结果，再计算$\tfrac{\partial w1}{\partial x}$并乘以前面的结果，即$((\tfrac{\partial y}{\partial w2})\tfrac{\partial w2}{\partial w1}\tfrac{\partial w1}{\partial x}$


下面仍以$z=x*y+\sin(x)$，求$\tfrac{\partial z}{\partial x}$为例，先看前向计算（forward pass）的过程，也叫前向传播（forward propagation）或正向传播、正向累积。首先将函数z的计算过程拆解为几个基础的运算，其中'?'表示待定，由实际输入决定。后面公式中的'?'也是相同的含义。

$$\begin{aligned}
& x = ? \\
& y = ? \\
& a = x * y \\
& b = sin(x) \\
& z = a + b
\end{aligned}
$$


根据链式法则，假设x,y前面还有输入变量t，对t求导步骤如下：
$$\begin{aligned}
& \tfrac{\partial x}{\partial t} = ? \\
& \tfrac{\partial y}{\partial t} = ? \\
& \tfrac{\partial a}{\partial t} = y\cdot\tfrac{\partial x}{\partial t}+x\cdot\tfrac{\partial y}{\partial t}\\
& \tfrac{\partial b}{\partial t} = cos(x)\cdot\tfrac{\partial x}{\partial t}\\
& \tfrac{\partial z}{\partial t} = \tfrac{\partial a}{\partial t} + \tfrac{\partial b}{\partial t}\\
\end{aligned}$$

令t=x，代入以上公式有$\text{ }\ \tfrac{\partial x}{\partial t}=\tfrac{\partial x}{\partial x}=1$，$\tfrac{\partial y}{\partial t}=0$。z关于x的偏导等价于z关于t的偏导$\tfrac{\partial z}{\partial x}=\tfrac{\partial z}{\partial t}$。计算代码如下，初始值x=1,y=1,dx=1,dy=0。

In [ ]:
import numpy as np
x = 1
y = 1
dx = 1   # dx/dt
dy = 0   # dy/dt
da = y * dx + x * dy  # da/dt
db = np.cos(x) * dx   # db/dt
dz = da + db          # dz/dt
print('forward mode auto-diff at (x=1,y=1):  dz =', dz)

梯度的前向模式可以跟输出z的计算过程同步计算，无需保存中间值，节省能存，但如果有多个输入，比如要计算输出z关于所有输入的偏导即$\tfrac{\partial z}{\partial x}$ 和$\tfrac{\partial z}{\partial y}$，需要先设置dx = 1 和dy = 0计算一遍，再设置dx = 0 和 dy = 1计算一遍。如果输入变量有n个，计算复杂度是O(n)。

再来看反向模式，仍然求函数$\text{ }\ z=x*y+\sin(x)$ 中z关于x的导数，假设输出z后面还做了一步运算得到s，即最终输出是s，各偏导的求导公式如下，大家可以基于函数z的数学式进行推导。
$$
\begin{aligned}
& \tfrac{\partial s}{\partial z} = ? \\
& \tfrac{\partial s}{\partial b} = \tfrac{\partial s}{\partial z} \tfrac{\partial z}{\partial b}= \tfrac{\partial s}{\partial z} \\
& \tfrac{\partial s}{\partial a} = \tfrac{\partial s}{\partial z} \tfrac{\partial z}{\partial a}=\tfrac{\partial s}{\partial z}\\
& \tfrac{\partial s}{\partial y} = x\cdot\tfrac{\partial s}{\partial a}\\
& \tfrac{\partial s}{\partial x} = y\cdot\tfrac{\partial s}{\partial a} + cos(x)\cdot\tfrac{\partial s}{\partial b}\\
\end{aligned}
$$

对应计算过程如下，令s=z，则有gz=$\tfrac{\partial s}{\partial z}$ = 1，令x=1,y=1，计算x=1,y=1处z关于x的偏导gx，跟前向计算的结果一致。如果要求z关于y的导数gy，不必再从头计算，而是复用中间的计算结果，计算复杂度更小。

In [ ]:
x = 1
y = 1
gz = 1   # ds/dz
gb = gz  # ds/db
ga = gz  # ds/da
gy = x * ga
gx = y * ga + np.cos(x) * gb
print('backward mode auto-diff at (x=1,y=1):  gx =',gx)

上面代码中基于反向模式的梯度计算是事先基于数学式推导出梯度计算的公式，工作量大，难以自动化。

另一种实现方式是基于计算图（computational graph），更直观。计算图将计算过程用有向无环图表示出来，通过节点（圆圈）和边（箭头）表示计算过程，节点代表执行的操作，将数据（输入、中间结果）写在箭头的上方，箭头的方向表示计算结果传递方向。计算图是现在各深度学习框架实现自动微分的主流方法。

计算图的优点之一是局部计算，无论全局是多么复杂的计算，只根据与当前节点相关的信息输出接下来的结果，使各个节点致力于简单的计算，从而简化问题。另一个优点是，利用计算图可以将中间的计算结果全部保存起来。

基于计算图的反向传播过程有以下几步：
1. 先构建正向计算图
2. 根据输入值，正向传播计算输出
3. 沿着与前向传播相反的方向，传递局部导数

下图是计算图中的单个节点，节点$f$代表z=f(x,y)的计算。

<img src="./notebook-imgs/node-f.png" width="200">

假设z后面还有一个计算节点，最终的输出是损失函数L，要求L关于x和y的导数。节点$f$的局部导数的传递过程如下图：将上游的导数值$\tfrac{\partial L}{\partial z}$乘以该节点的局部导数，分别是$\tfrac{\partial z}{\partial x}$和$\tfrac{\partial z}{\partial y}$，然后将结果传递给下一个节点。通过反向传播计算的结果是$\tfrac{\partial L}{\partial x}$ = $\tfrac{\partial L}{\partial z} \tfrac{\partial z}{\partial x}$。如果L=z，则$\tfrac{\partial L}{\partial z}=1$。

<!-- <img src="./notebook-imgs/node-backprop.png" width="600"> -->



<img src="./notebook-imgs/bk-f.png" width="200">

加法节点计算图及反向传播：左边是正向传播，右边是反向传播。其中$\tfrac{\partial L}{\partial z}$是上游节点传过来的局部导数，L指的是损失函数的输出。

<img src="./notebook-imgs/add-backprop.png" width="400">

乘法节点计算图及反向传播：左边是正向传播，右边是反向传播

<img src="./notebook-imgs/mul-backprop.png" width="400">

下面以$\text{ }\ z=x*y+\sin(x)$为例，手推基于计算图的反向传播梯度计算：

（1）先画出计算图

<img src="./notebook-imgs/compute-graph.jpg" width="400">

（2）代入输入数据，正向传播求解输出

<img src="./notebook-imgs/compute-graph-forward.jpg" width="400">

（3）反向传播，图中红线所示，输出端初始化为1。由于变量x有两条路径，两条路径的偏导相加，求得$\tfrac{\partial z}{\partial x}$ = 1+0.5403 =1.5403，与前面的计算结果一致。

<img src="./notebook-imgs/compute-graph-backward.jpg" width="400">

本节没有需要编码实现的，只需要执行以上cell块，熟悉原理和实现。